In [16]:
import tensorflow as tf
import pandas as pd
import numpy as np

tf.keras.backend.clear_session()

from kfold_and_metrics import *

In [17]:
df = pd.read_csv("test.csv")


df = df.drop(columns=['diagnostics_Mask-original_Spacing',
       'diagnostics_Mask-original_Size',
       'diagnostics_Mask-original_BoundingBox',
       'diagnostics_Mask-original_CenterOfMassIndex',
       'diagnostics_Mask-original_CenterOfMass',
       'diagnostics_Image-interpolated_Spacing',
       'diagnostics_Image-interpolated_Size',
       'diagnostics_Mask-interpolated_Spacing',
       'diagnostics_Mask-interpolated_Size',
       'diagnostics_Mask-interpolated_BoundingBox',
       'diagnostics_Mask-interpolated_CenterOfMassIndex',
       'diagnostics_Mask-interpolated_CenterOfMass',
       'original_shape_Elongation', 'original_shape_Flatness',
       'original_shape_LeastAxisLength', 'original_shape_MajorAxisLength',
       'original_shape_MinorAxisLength','id','Id'])
to_delete = [*df.columns[24:28], df.columns[33]]
df = df.drop(columns=to_delete)

df.head()

C:\Users\anton\AppData\Local\Temp\ipykernel_30152\2315874082.py:1: DtypeWarning: Columns (24,25,26,27,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("test.csv")


,diagnostics_Image-original_Mean,diagnostics_Mask-original_VoxelNum,diagnostics_Mask-original_VolumeNum,diagnostics_Image-interpolated_Mean,diagnostics_Image-interpolated_Minimum,diagnostics_Image-interpolated_Maximum,diagnostics_Mask-interpolated_VoxelNum,diagnostics_Mask-interpolated_VolumeNum,diagnostics_Mask-interpolated_Mean,diagnostics_Mask-interpolated_Minimum,...,calcification,sphericity,margin,lobulation,spiculation,texture,diameter,surface_area,volume,malignancy
0,-826.943929,5905,1,-417.494203,-990.291016,1038.270874,909,2,-139.445774,-878.871155,...,6,3,3,3,4,5,32.205888,2801.428431,7869.369507,1
1,-826.943929,4613,1,-405.581777,-982.456726,949.768005,699,1,-19.452085,-825.718018,...,6,4,4,5,5,5,33.188692,2198.454852,6146.438599,1
2,-826.943929,4955,1,-410.236759,-990.291016,1038.270874,772,1,-60.709724,-809.482544,...,6,3,2,3,3,5,31.959332,2392.704188,6594.474792,1
3,-826.943929,5498,1,-416.576321,-990.291016,1038.270874,841,2,-94.890616,-809.482544,...,6,5,4,1,5,4,33.669337,2573.278820,7348.411560,1
4,-826.943929,10351,1,-546.359139,-1007.657349,1020.174988,749,1,-769.283626,-976.774536,...,6,5,1,1,1,1,30.023172,2880.758652,6576.900681,1


In [ ]:
activation_possibilities = ["softmax", "relu", "sigmoid"]
max = 0
best = []
for activation_hidden in activation_possibilities:
    for num_nodes_hidden_layer in range(5, len(df.columns)*2//3, 10):
        for gradient in [0.001, 0.003, 0.005, 0.007, 0.01]:
            
            nn_model = tf.keras.models.Sequential([
                tf.keras.layers.Input((len(df.columns)-2,), name="input"),
                tf.keras.layers.Dense(num_nodes_hidden_layer,activation=activation_hidden),
                tf.keras.layers.Dense(2,activation='softmax')
            ])
            nn_model.compile(
                optimizer=tf.keras.optimizers.SGD(gradient), 
                loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
            )
            score = k_fold_cv_keras(compiled_model=nn_model, df=df)
            results = mean_std_results_k_fold_CV(score)

            print(results)

            accuracy_avg = results.iloc[1,1]
            if accuracy_avg > max:
                best = [num_nodes_hidden_layer, activation_hidden, gradient]



nn_model = tf.keras.models.Sequential([
    tf.keras.layers.Input((len(df.columns)-2,)),
    tf.keras.layers.Dense(best[0],activation=best[1]),
    tf.keras.layers.Dense(2,activation='softmax')
])
            
nn_model.compile(
    optimizer=tf.keras.optimizers.SGD(best[2]), 
    loss=tf.keras.losses.SparseCategoricalCrossentropy(), 
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

In [ ]:
cv_results = k_fold_cv_keras(compiled_model=nn_model, df=df)
metrics = mean_std_results_k_fold_CV(cv_results)
metrics